In [1]:
import cv2
import uuid


def color_match(image1, image2):
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
    
    img1_lab = cv2.cvtColor(img1, cv2.COLOR_BGR2LAB)
    img2_lab = cv2.cvtColor(img2, cv2.COLOR_BGR2LAB)

    # Calculate mean and standard deviation for each channel in LAB color space
    img1_mean, img1_std = cv2.meanStdDev(img1_lab)
    img2_mean, img2_std = cv2.meanStdDev(img2_lab)

    # Perform color matching by adjusting mean and standard deviation
    for i in range(3):  # Iterate over L, A, B channels
        img1_lab[:,:,i] = img1_lab[:,:,i] - img1_mean[i]
        img1_lab[:,:,i] = img1_lab[:,:,i] * (img2_std[i] / img1_std[i])
        img1_lab[:,:,i] = img1_lab[:,:,i] + img2_mean[i]

    # Convert back to BGR color space
    matched_img = cv2.cvtColor(img1_lab, cv2.COLOR_LAB2BGR)

    return matched_img

# Example usage
#image1- targeImage
#image2 - palletImage
target_image_path ='targetimage.jpeg'
pallet_image_path = 'palletimage.jpeg'
matched_image = color_match(target_image_path, pallet_image_path)

# Display the matched image
# cv2.imshow('Color Matched Image', matched_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
output_path = str(uuid.uuid4())+'_matched_image.jpg'
cv2.imwrite(output_path, matched_image)
print(f"Color matched image saved to {output_path}")


Color matched image saved to f9c56401-f37b-483d-8b8a-086a19615f57_matched_image.jpg


In [2]:
import cv2

# Load the image
image = cv2.imread('targetimage.jpeg')

# Define the dimensions of each part
num_rows = 3  # Number of rows for splitting
num_cols = 3  # Number of columns for splitting
height, width, _ = image.shape

# Calculate the dimensions of each part
part_height = height // num_rows
part_width = width // num_cols

# Initialize an empty list to store the parts
image_parts = []

# Iterate through rows and columns to split the image
for row in range(num_rows):
    for col in range(num_cols):
        # Calculate the starting and ending coordinates for each part
        start_row = row * part_height
        end_row = (row + 1) * part_height
        start_col = col * part_width
        end_col = (col + 1) * part_width
        
        # Extract the part from the image
        part = image[start_row:end_row, start_col:end_col]
        
        # Append the part to the list
        image_parts.append(part)
        
for i, part in enumerate(image_parts):
    cv2.imwrite(f'part_{i+1}.jpg', part)


In [7]:
import cv2
import numpy as np

# Load all the image parts
image_parts = []
for i in range(1, num_rows*num_cols + 1):  # Assuming num_rows and num_cols are defined
    #part = cv2.imread(f'part_{i}.jpg')
    part = color_match(f'part_{i}.jpg', 'palletimage.jpeg')
    image_parts.append(part)

# Determine the dimensions of the combined image
combined_height = num_rows * part_height
combined_width = num_cols * part_width

# Create an empty canvas for the combined image
combined_image = np.zeros((combined_height, combined_width, 3), dtype=np.uint8)

# Combine the image parts into the final image
for i, part in enumerate(image_parts):
    row = i // num_cols
    col = i % num_cols
    start_row = row * part_height
    end_row = (row + 1) * part_height
    start_col = col * part_width
    end_col = (col + 1) * part_width
    combined_image[start_row:end_row, start_col:end_col] = part

# Optionally, save the combined image to a file
cv2.imwrite('combined_image.jpg', combined_image)


True

In [6]:
mth_jpg = color_match('part_1.jpg', 'palletimage.jpeg')
output_path = str(uuid.uuid4())+'_matched_image.jpg'
cv2.imwrite(output_path, mth_jpg)
print(f"Color matched image saved to {output_path}")

Color matched image saved to 2298fc3a-1a36-49a2-ae75-bd85a363fa7a_matched_image.jpg


In [10]:
import cv2
import numpy as np
import os
from sklearn.cluster import MiniBatchKMeans

# Load the large image
large_image = cv2.imread('large_image.jpeg')

# Directory containing tile images
tile_dir = 'tiles_images/'
tile_images = [os.path.join(tile_dir, img) for img in os.listdir(tile_dir)]

# Resize tile images to a standard size (e.g., 50x50 pixels)
tile_size = (50, 50)
resized_tiles = [cv2.resize(cv2.imread(img), tile_size) for img in tile_images]

# Flatten and reshape tile images for k-means clustering
reshaped_tiles = [tile.reshape(-1, 3) for tile in resized_tiles]

# Number of clusters for k-means (adjust as needed)
num_clusters = 5

# Perform k-means clustering on tile colors
kmeans = MiniBatchKMeans(n_clusters=num_clusters)
kmeans.fit(np.vstack(reshaped_tiles))

# Iterate over each region in the large image (e.g., using a sliding window)
window_size = tile_size  # Adjust window size as needed
mosaic = np.zeros_like(large_image)

for y in range(0, large_image.shape[0], window_size[1]):
    for x in range(0, large_image.shape[1], window_size[0]):
        region = large_image[y:y + window_size[1], x:x + window_size[0]]
        region_features = region.reshape(-1, 3)
        
        # Predict the cluster for the region using k-means
        cluster = kmeans.predict(region_features)
        
        # Compute distances between region pixels and cluster centroids
        distances = []
        for pixel in region_features:
            pixel_distances = np.linalg.norm(kmeans.cluster_centers_ - pixel, axis=1)
            distances.append(np.min(pixel_distances))
        
        # Find the best matching tile image based on minimum distance
        if distances:
            best_tile_index = np.argmin(distances)
            if 0 <= best_tile_index < len(resized_tiles):
                best_tile = resized_tiles[best_tile_index]
            else:
                # Handle index out of range by selecting a default tile (e.g., first tile)
                best_tile = resized_tiles[0]
        else:
            # Handle empty distances list by selecting a default tile (e.g., first tile)
            best_tile = resized_tiles[0]
        
        # Replace the region in the mosaic with the best matching tile
        mosaic[y:y + window_size[1], x:x + window_size[0]] = best_tile


# Save the mosaic image
cv2.imwrite('mosaic.jpg', mosaic)

# # Display the mosaic image
# cv2.imshow('Photo Mosaic', mosaic)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


ValueError: could not broadcast input array from shape (50,50,3) into shape (50,36,3)

In [2]:
import cv2

def split_images(target_image_path,dir, rows=3, columns=3):
    image_parts = []
    target = cv2.imread(target_image_path)
    height, width, _ = target.shape
    part_height = height // rows
    part_width = width // columns
    for row in range(rows):
        for col in range(columns):
            start_row = row * part_height
            end_row = (row + 1) * part_height
            start_col = col * part_width
            end_col = (col + 1) * part_width

            part = target[start_row:end_row, start_col:end_col]
            image_parts.append(part)
                
        for i, part in enumerate(image_parts):
            cv2.imwrite(f'{dir}/part_{i+1}.jpg', part)

In [3]:
split_images('large_image.jpeg', 'output', 5, 5)

In [28]:
import cv2
import numpy as np
import os
from sklearn.cluster import MiniBatchKMeans

# Load the large image
large_image = cv2.imread('large_image.jpeg')

# Directory containing tile images
tile_dir = 'tiles_images/'
tile_images = [os.path.join(tile_dir, img) for img in os.listdir(tile_dir)]


In [34]:
tile_images

['tiles_images/pallet3.jpeg',
 'tiles_images/pallet2.jpeg',
 'tiles_images/pallet5.jpeg',
 'tiles_images/pallet4.jpeg',
 'tiles_images/pallet8.jpeg',
 'tiles_images/pallet7.jpeg',
 'tiles_images/pallet6.jpeg']

In [29]:
# Resize tile images to a standard size (e.g., 50x50 pixels)
tile_size = (50, 50)
resized_tiles = [cv2.resize(cv2.imread(img), tile_size) for img in tile_images]


In [30]:
# Flatten and reshape tile images for k-means clustering
reshaped_tiles = [tile.reshape(-1, 3) for tile in resized_tiles]

# Number of clusters for k-means (adjust as needed)
num_clusters = 5

# Perform k-means clustering on tile colors
kmeans = MiniBatchKMeans(n_clusters=num_clusters)
kmeans.fit(np.vstack(reshaped_tiles))


/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


MiniBatchKMeans(n_clusters=5)

In [31]:
def color_match_and_blend(region, tile):
    # Resize the tile to match the size of the region
    tile_resized = cv2.resize(tile, (region.shape[1], region.shape[0]))
    
    # Compute the average color of the region and resized tile
    region_avg_color = np.mean(region, axis=(0, 1))
    tile_avg_color = np.mean(tile_resized, axis=(0, 1))
    
    # Calculate the color difference between region and resized tile
    color_diff = np.linalg.norm(region_avg_color - tile_avg_color)
    
    # Blend the region and resized tile based on color similarity
    alpha = 1.0 - (color_diff / 255.0)  # Adjust alpha based on color difference
    blended_region = cv2.addWeighted(region, alpha, tile_resized, 1.0 - alpha, 0)
    
    return blended_region

In [32]:
# Iterate over each region in the large image (e.g., using a sliding window)
window_size = tile_size  # Adjust window size as needed
mosaic = np.zeros_like(large_image)

for y in range(0, large_image.shape[0], window_size[1]):
    for x in range(0, large_image.shape[1], window_size[0]):
        region = large_image[y:y + window_size[1], x:x + window_size[0]]
        #print(region.shape)
        region_avg_color = np.mean(region, axis=(0, 1))  # Compute average color of the region
        
        # Find the best matching tile image based on average color difference
        min_diff = float('inf')
        best_tile = None
        for tile in resized_tiles:
            tile_avg_color = np.mean(tile, axis=(0, 1))
            color_diff = np.linalg.norm(region_avg_color - tile_avg_color)
            if color_diff < min_diff:
                min_diff = color_diff
                best_tile = tile
        
        # Replace the region in the mosaic with the best matching tile
        blended_region = color_match_and_blend(region, best_tile)    
        mosaic[y:y + window_size[1], x:x + window_size[0]] = cv2.resize(blended_region,(region.shape[1], region.shape[0]))


In [33]:
cv2.imwrite('mosaicr4lgk.jpg', mosaic)

True